In [1]:
import xarray as xr
from pathlib import Path
from typing import Iterable, List, Optional, Tuple
import numpy.typing as npt
import matplotlib.pyplot as plt
import numpy as np
import imageio.v2 as imageio
from grid import file_to_date
def load_netcdfs(paths: Iterable[Path]) -> List[xr.Dataset]:
    """Loads a list of netCDF files"""
    return [xr.open_dataset(p.as_posix(),engine="netcdf4") for p in paths]

In [2]:
# Resolution
RES = "01d"
res = "1"

In [3]:
def plot_grid(
    x_grid: npt.NDArray[np.float64],
    y_grid: npt.NDArray[np.float64],
    grid: npt.NDArray[np.float64],
    datavar: str,
    lon_boundary: Optional[Tuple[float, float]] = None,
    lat_boundary: Optional[Tuple[float, float]] = None,
    points_lon: Optional[npt.NDArray[np.float64]] = None,
    points_lat: Optional[npt.NDArray[np.float64]] = None,
    figsize: Tuple[float, float] = (10, 3),
    cmap: str = "jet",
    vmin: Optional[float] = None,
    vmax: Optional[float] = None,
    title: str = None,
    pic_name: str = None
) -> None:
    # Input handling
    if lon_boundary is None:
        lon_boundary = [x_grid[0,:].min(), x_grid[0,:].max()]
    if lat_boundary is None:
        lat_boundary = [y_grid[:,0].min(), y_grid[:,0].max()]
    if vmin is None:
        vmin = grid.min()
    if vmax is None:
        vmax = grid.max()

    # Extent of the grid
    extent = [*lon_boundary, *lat_boundary]

    # Valid points in the choosen boundary
    if points_lon is not None and points_lat is not None:
        points_in_zone = (
            (points_lon >= extent[0])
            & (points_lon <= extent[1])
            & (points_lat >= extent[2])
            & (points_lat <= extent[3])
        )
    
    # Plot
    fig = plt.figure(figsize=figsize)
    plt.imshow(grid, extent=extent, origin='lower', vmin=vmin, vmax=vmax, cmap=cmap)
    cbar = plt.colorbar()
    if points_lon is not None and points_lat is not None:
        plt.scatter(points_lon[points_in_zone], points_lat[points_in_zone], s=1, color=(0,0,0,0.01))
    plt.xlim(lon_boundary)
    plt.ylim(lat_boundary)
    plt.xlabel(f"Longitude [\N{DEGREE SIGN}]")
    plt.ylabel(f"Latitude [\N{DEGREE SIGN}]")
    plt.title(title)
    cbar.set_label(f"{datavar} [m]")
    Path(f"Grids/{RES}/figs/{datavar}").mkdir(parents=True,exist_ok=True)
    plt.savefig(Path(f"Grids/{RES}/figs/{datavar}/{pic_name}.png"))
    plt.close()
    # plt.show()

In [4]:
grid_paths = Path(f'Grids/{RES}/3days').glob('2004*.nc')
grids = load_netcdfs(grid_paths)
# raw_paths = [Path(f'Processed','all',path) for path in ['2005_1_17.nc','2005_1_18.nc','2005_1_19.nc']]
# raw = load_netcdfs(raw_paths)
# raw = xr.concat(raw,dim="time")
# grid_day = load_netcdfs([Path(f'Grids/{RES}/3days/2005_1_18_10m.nc')])

In [5]:
np.nanmedian(grids[0].wind_speed.data)

7.266554636335684

In [6]:
cbar: dict[str,list[float]] = {
    "sla": [-.75,.75],
    "sst": [0,25],
    "swh": [0,5],
    "wind_speed": [0,12]
}
for path,grid in zip(Path(f'Grids/{RES}/3days').glob('*.nc'),grids):
# for path,grid in zip(raw_paths, grid_day):
    titlename = "-".join(path.name.split(".")[0].split("_")[:3])
    filename = "_".join(path.name.split(".")[0].split("_")[:3])
    for datavar in grid.data_vars:
        plot_grid(
            x_grid=grid.lons.values, 
            y_grid=grid.lats.values, 
            grid=grid[datavar].values,
            datavar=datavar,
            # points_lon = raw.lon.values,
            # points_lat = raw.lat.values,
            lon_boundary=(-180,180), 
            lat_boundary=(-80,80),
            figsize=(15,5),
            vmax=cbar[datavar][0], 
            vmin=cbar[datavar][1],
            title=f"{datavar} {titlename}\n3 day moving window - {res}\N{DEGREE SIGN} grid resolution",
            pic_name=f"{filename}"
            # pic_name=filename
        )

In [7]:
for datavar in grids[0].data_vars:
    images = []
    basepath = Path(f'Grids/{RES}/figs/{datavar}')
    grid_paths = basepath.glob('*.png')

    dates = []
    for file in grid_paths:
        dt = file_to_date(file)
        dates.append(dt)
    dates.sort()

    for d in dates:
        filename = f"{basepath.as_posix()}/{d.year}_{d.month}_{d.day}.png"
        images.append(imageio.imread(filename))
    imageio.mimsave(f'Grids/{RES}/figs/{datavar}/{d.year}.gif', images)